In [3]:

# ==================================================================================================
# load TiSR module
# ==================================================================================================
#using Revise
using TiSR
using ArgParse
using DataFrames
using CSV
import DynamicQuantities: uparse,  @u_str
using Statistics
println(pathof(TiSR))


# ==================================================================================================
# read arguments from the command line 
# ==================================================================================================
function parse_commandline()
    s = ArgParseSettings()
    @add_arg_table s begin
        "dataset"
            arg_type = Int
            help = "options are: 1,2,3,4,5,6,7,8,9" #1: Hubble, 2: Kepler, 3: Newton, 4: Planck, 5: Leavitt, 6: Schechter, 7: Bode, 8: Ideal gas, 9: Rydberg
        "dataset_type"
            arg_type = String
            help = "options are: original, wo_noise"
        "obj"
            arg_type = String
            help = "options are: f1f2f4, f1f2f3, f1f3f4"
        "algo"
            help = "options are: correct_all, correct_mut, obj_penalty, death_penalty, baseline"
            arg_type = String   
        "run"
            arg_type = Int
    end

    return parse_args(s)
end

parsed_args = parse_commandline()
#Instead of parsing the arguments, I just define them hardcoded here for Planck test case
algo = "death_penalty"
dataset = 4
dataset_type = "wo_noise"
obj = "f1f2f4"
run = 1

has_original_measures = [1,2,5,7] #original datasets available

if dataset_type == "original"
    if dataset in has_original_measures
        dataset_appendix = "_original"
    else
        dataset_appendix = "_generated_noise"
    end
elseif dataset_type == "wo_noise"
    dataset_appendix = "_generated_wo_noise"
end
dataset_prefix = "./data/empiricalBench"
arbitrary_name = "$(dataset)_$(dataset_appendix)_$(obj)_$(algo)_(run)"
#input units, target unit, dimensionless
if dataset == 1 #Hubble, stays as is
    data_matrix = Matrix(CSV.read("$dataset_prefix/hubble$dataset_appendix.csv", DataFrame))
    units = [u"m",  u"km * s^-1", u"0"] #use m instead of Mpc
elseif dataset == 2 #Kepler, stays as is
    data_matrix = Matrix(CSV.read("$dataset_prefix/kepler$dataset_appendix.csv", DataFrame))
    units = [u"m", u"day", u"0"] #use m instead of AU 
elseif dataset == 3 #Newton
    data_matrix = Matrix(CSV.read("$dataset_prefix/newton$dataset_appendix.csv", DataFrame))
    units = [u"m", u"kg", u"kg", u"N", u"0"]
elseif dataset == 4 #Planck, first try to remove scaling using exp(y), might change to log only later
    data_matrix = Matrix(CSV.read("$dataset_prefix/planck$(dataset_appendix)_scaled.csv", DataFrame))
    #data_matrix[:, end] .= log.(data_matrix[:, end])
    units = [u"Hz", u"K", u"W * m^(-2) * Hz^(-1)", u"0"]
elseif dataset == 5 #Leavitt, provide with log10 instead of log
    data_matrix = Matrix(CSV.read("$dataset_prefix/leavitt$dataset_appendix.csv", DataFrame))
    units = [u"0", u"0", u"0"]
elseif dataset == 6 #Schechter, uses log scale and log(x1), therefore unit of x1 is set to dimensionless
    data_matrix = Matrix(CSV.read("$dataset_prefix/schechter$dataset_appendix.csv", DataFrame))
    units = [u"0", u"0", u"0"]
elseif dataset == 7 #Bode, dataset not log scaled, therefore stays as is
    data_matrix = Matrix(CSV.read("$dataset_prefix/bode_original.csv", DataFrame))
    units = [u"0", u"0", u"0"]
elseif dataset == 8 #Ideal gas, remove log scaling using exp(y)
    data_matrix = Matrix(CSV.read("$dataset_prefix/ideal_gas$dataset_appendix.csv", DataFrame))
    data_matrix[:, end] .= exp.(data_matrix[:, end])
    units = [u"mol", u"K", u"m^3", u"Pa", u"0"]
elseif dataset == 9 #Rydberg, uses log scale but input features are dimensionless, therefore stays as is
    data_matrix = Matrix(CSV.read("$dataset_prefix/rydberg$dataset_appendix.csv", DataFrame))
    units = [u"0", u"0", u"0", u"0"]
end

if algo == "correct_all"
    always_correct_dims = true
    p_correct_dims = 0.0
    death_penalty_dims = false
elseif algo == "correct_mut"
    always_correct_dims = false
    p_correct_dims = 4.0
    death_penalty_dims = true
elseif algo == "obj_penalty"
    always_correct_dims = false
    p_correct_dims = 0.0
    death_penalty_dims = false
elseif algo == "death_penalty"
    always_correct_dims = false
    p_correct_dims = 0.0
    death_penalty_dims = true
elseif algo == "baseline"
    always_correct_dims = false
    p_correct_dims = 0.0
    death_penalty_dims = false
end

if obj == "f1f2f3"
    hall_of_fame_objectives = [:ms_processed_e, :dim_penalty, :compl]
    selection_objectives = [:ms_processed_e, :minus_spearman, :dim_penalty]
elseif obj == "f1f2f4"
    hall_of_fame_objectives = [:ms_processed_e, :compl]
    selection_objectives = [:ms_processed_e, :minus_spearman, :compl]
elseif obj == "f1f3f4"
    hall_of_fame_objectives = [:ms_processed_e, :dim_penalty, :compl]
    selection_objectives = [:ms_processed_e, :dim_penalty, :compl]
end

# ==================================================================================================
# preparation
# ==================================================================================================
parts = [0.8, 0.2]

# ==================================================================================================
# options -> specify some custom settings, where the default setting is unsatisfactory
# ==================================================================================================
pow_abs(v1, v2) = abs(v1)^v2
sqrt_abs(v1) = sqrt(abs(v1))
pow2(v1) = v1^2
pow3(v1) = v1^3

ops, data = Options(
    data_descript=data_descript(
        data_matrix;
        arbitrary_name = arbitrary_name,
        parts          = parts,
       #fit_weights    = fit_weights, 
        units          = [u"m", u"0", u"0"],
    ),
    general=general_params(
        n_gens          = typemax(Int64),
        pop_size        = 500,
        max_compl       = 30,
        pow_abs_param   = true,
        prevent_doubles = 1e-50,
        t_lim           = 60 * 60.0, #make this 1 hour
        multithreadding = true,
        always_correct_dims = always_correct_dims,
        death_penalty_dims  = death_penalty_dims,
    ),
    selection=selection_params(
        hall_of_fame_objectives           = hall_of_fame_objectives,          # -> objectives for the hall_of_fame
        selection_objectives              = selection_objectives              # -> objectives for the Pareto-optimal selection part of selection
    ),
    fitting=fitting_params(
        early_stop_iter = 5,
        max_iter        = 30,
        lasso_factor    = 1e-7),
    binops         = (  +,   -,   *,   /,  ^),  # -> binary function set to choose from
    p_binops       = (1.0, 1.0, 1.0, 1.0, 0.0),  # -> probabilites for selection of each binary functions (same length as provided binops) (dont need to add up to 1, adjusted accordingly)
    unaops         = (exp, log, sin, cos, abs, sqrt_abs, pow2, pow3),  # -> unary function set to choose from
    p_unaops       = (1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0),  # -> probability for each unary function
    illegal_dict = Dict(:sin => (sin, cos),
                    :cos => (sin, cos),
                    :abs => (abs,),
                    :exp => (exp, log, sqrt_abs),
                    :log => (exp, log),
                    :pow2 => (log, sqrt_abs),
                    :sqrt_abs => (pow2,),),

    mutation=mutation_params(;
        p_crossover        = 4.0,
        p_point            = 0.5,
        p_innergrow        = 0.0,
        p_insert           = 0.2,
        p_hoist            = 0.2,
        p_subtree          = 0.2,
        p_add_term         = 0.1,
        p_simplify         = 0.5,
        p_drastic_simplify = 0.5,
        p_correct_dims     = p_correct_dims,),
    
    
);



/Users/juliareu/GitCode/TiSR/src/TiSR.jl


┌ Warning: a low prevent_doubles may not detect equal individuals
└ @ TiSR /Users/juliareu/GitCode/TiSR/src/options.jl:106
┌ Warning: ^ is only valid for positive bases. Otherwise provide a pow(abs(x), y) function with its own drawbacks.
└ @ TiSR /Users/juliareu/GitCode/TiSR/src/options.jl:39


In [3]:
# copy_data_matrix = log.(data_matrix)
copy_data_matrix = data_matrix

v1 = copy_data_matrix[:, 1]
v2 = copy_data_matrix[:, 2]

# func = p -> @. log(abs(p[1] * pow3(v1)/(exp(p[2]*v1/v2) - 1.0)))
# func = p -> @. p[1] + 3 * log(v1) - log(exp(p[2] * v1/v2) - 1)
# func = p -> @. p[1] * pow3(v1) * (1/(exp(p[2] * v1 / (v2))-p[3]))

func = p -> @. (pow2((p[1]*v1))*(v2-(v1*(p[3]-exp(((p[2]*v1)/v2))))))


minim = p -> mean(abs2, func(p) .- copy_data_matrix[:, 3])

using Optim
using LineSearches

#True parameter values are: 1.4744994402842404e-50, 4.79924466221135e-11

p0 = [
    0.000175
    1.92e-12
    1.0
]

res = Optim.optimize(
    minim,
    p0,
    Optim.LBFGS(;linesearch=LineSearches.BackTracking()),
    Optim.Options(),
    autodiff=:forward,
)


p_best = Optim.minimizer(res)
Optim.minimum(res)

BoundsError: BoundsError: attempt to access 8×2 Matrix{Float64} at index [1:8, 3]

In [4]:
6.62607004e-9/1.38064852e-3

4.79924466221135e-6

In [5]:
# Planck
#Die richtige Gleichung mit falschen Startwerten der Parameter
#True parameter values are: 1.4744994402842404e-50, 4.79924466221135e-11
eq1 = string_to_node("(1.4744994402842404e-50 * pow3(v1)/(exp(4.79924466221135e-11*v1/v2)-1))", ops)
eq2 = string_to_node("(1.47449944028424e-12 * pow3(v1) * (1/(exp(4.79924466221135e-6 * v1 / (v2))-1)))", ops)

#Eine Gleichung, die der Algorithmus nach einer Stunde Laufzeit gefunden hat, stimmt fast, bis auf -1 im Nenner fehlt
eq3 = string_to_node("(pow3((3.08811211513492e-17*v1))/exp((6.12814427966739e-11/(v2/v1))))", ops)
eq4 = string_to_node("(pow2((0.000175*v1))*(v2-(v1*(1.0-exp(((1.92e-12*v1)/v2))))))", ops)


#Die logarithmierte Gleichung
eq5 = string_to_node("-114.74093608002615 + 3.0*log(v1) - log(exp(4.79924466221135e-11*v1/v2) - 1.0)", ops)

eq = eq5
list_of_param = list_of_param_nodes(eq)
print(eq)
res = residual_after_fitting(eq, data, ops, list_of_param)
println("Residuals  : ",res[1])
println("Valid?     : ",res[2])

println("MSE        : ", mean(abs2, res[1]))

((-115.0+(3.0*log(v1)))-log((exp(((4.8e-11*v1)/v2))-1.0)))


Residuals  : [1123.7791109163968, 136.57132313532736, 89.68909683045675, 595.7978427440013, 89.53063798372945, 97.38265064260035, 93.78251375243737, 90.7673706190088, 92.03681523412077, 91.71075677890039, 89.76652056715695, 108.2740310288816, 117.76357886626113, 214.019343706269, 89.34314321983467, 91.84935698085101, 89.6402892424658, 90.06848770277966, 83.58195710153674, 95.14911167904741, 88.56398734344525, 116.38735175608414, 89.64679203221866, 95.02265715415945, 87.08111462925073, 89.343222330244, 90.03925635760334, 123.67441546960758, 89.65140405976622, 90.05283697678317, 85.81346805375485, 92.46425555605119, 96.35027439105494, 91.02779853398108, 467.16701502794103, 91.7436505581647, 90.06216989836314, 90.0228923466205, 91.80252728066759, 106.92738820465496, 133.2452294016874, 85.48573147338577, 89.3756144202173, 108.35728069111829, 89.63814405483656, 120.70407960521882, 90.85980484911902, 96.57443297361205, 92.23053760926715, 83.49456292557836, 220.53180607130605, 90.881222313642

In [43]:
# Leavitt
eq = string_to_node("16.084 - log(v1 + 1.65)", ops)
eq2 = string_to_node("(373.0/(24.0+sqrt_abs(v1)))", ops)

#(16.115046924756033 - log(x0 + 1.624759616856815))
eq3 = string_to_node("log((9.25484803213189e6/(1.324347637779+v1)))", ops)
eq4 = string_to_node("(16.023109856068-log((1.14627398447791+v1)))", ops)
eq5 = string_to_node("log((8.86528648616603e6/(1.04597046671338+v1)))", ops)
list_of_param = list_of_param_nodes(eq4)
res = residual_after_fitting(eq4, data, ops, list_of_param)
println("Residuals  : ",res[1])
println("Valid?     : ",res[2])

println("MSE        : ", mean(abs2, res[1]))

Residuals  : [-0.3110714198495863, -0.16577829614656459, -0.13852902764259412, 0.21726091131572112, -0.03861267776633781, -0.1110228022275912, -0.22284253102017892, 0.19908956036633008, 0.25576829194890394, 0.02144893504729417, 0.09771216475480493, 0.22895541303074118, 0.13643602249329234, -0.2335456834409655, 0.1107662485208234, 0.11082605185035632, 0.10458646565762564, -0.03234935237720293, -0.08316838430691043, 0.34584484202505195, 0.01572668453429138, 0.06000983702450924, -0.18122628148501008, -0.3645718539895899, -0.4604733652117048, -0.004319287830085727]
Valid?     : true
MSE        : 0.0399799046506052


In [ ]:
# bode_original
eq = string_to_node("log(1.0 + v1)", ops)

In [17]:
using SymbolicUtils
eq = string_to_node("(-0.00119*(pow2((-0.00876*v1))/(-1.0+pow2((v1/v2)))))", ops)
eq = node_to_symbolic(eq, ops)
eq = SymbolicUtils.simplify(eq)
print(eq)

(-9.131774400000002e-8(v1^2)*(v2^2)) / (v1^2 - (v2^2))

In [18]:
eq = string_to_node("(-0.00119*(pow2((-0.00876*v1))/(-1.0+pow2((v1/v2)))))", ops)
simplify_w_symbolic_utils!(eq, ops)
print(eq)

(((-9.13e-8*(v1^2.0))*(v2^2.0))/((v1^2.0)+(-1.0*(v2^2.0))))


In [12]:
using Symbolics
using SymbolicUtils
@variables v1 v2 v3 v4 v5
pow_abs(v1, v2) = abs(v1)^v2
sqrt_abs(v1) = sqrt(abs(v1))
pow2(v1) = v1^2
pow3(v1) = v1^3

function simplify_equation(eq)
    #eq = replace(eq, "--" => "+")
    eq_simp = eval(Meta.parse(eq))
    eq_simp = PolyForm(eq_simp, recurse = true)
    simplify(eq_simp)
    #eq_simp = PolyForm(eq_simp, recurse = true)

    #repr(eq_simp)
end   



eq = " (exp((-1.57e-11v1) / (1.0 + v2) + log(6.24e-15v1))^3) / sqrt(abs(v1)) "
@variables v1 v2 v3 v4 v5
#Q1 = eval(Meta.parse(eq))
simplify_equation(eq)


exp((3(v2*log(6.24e-15v1) + log(6.24e-15v1) - 1.57e-11v1)) / (1.0 + v2)) / sqrt(abs(v1))

In [25]:
import SymPy

@SymPy.syms v1::Positive v2::Positive v3::Positive v4::Positive v5::Positive
eq = " (exp((-1.57e-11v1) / (1.0 + v2) + log(6.24e-15v1))^3) / sqrt(abs(v1)) "
eq = SymPy.parse(Sym, eq)
eq1 = SymPy.simplify(eq)
pretty_expr = SymPy.pretty(eq1)

println(pretty_expr)

MethodError: MethodError: no method matching parse(::Type{Sym}, ::String)

Closest candidates are:
  parse(!Matched::Type{P}, ::AbstractString; kwargs...) where P<:FilePathsBase.AbstractPath
   @ FilePathsBase ~/.julia/packages/FilePathsBase/4RrDh/src/path.jl:77
  parse(!Matched::Type{Sockets.IPAddr}, ::AbstractString)
   @ Sockets /opt/homebrew/Cellar/julia/1.9.3/share/julia/stdlib/v1.9/Sockets/src/IPAddr.jl:246
  parse(!Matched::Type{T}, ::AbstractString) where T<:Complex
   @ Base parse.jl:386
  ...


In [1]:
log(1.90752378459450e-32)

-73.03691702272278

In [18]:
using Distributions
using DataFrames    
using CSV
#f(v1, v2, v3) .= v1./v2 .+ 0.35 .*v3 .+ 10.0 .*log((v1 .^2) ./v3)

v1 = [rand(Uniform(5.0, 10.0)) for i in 1:50]
println(v1)
v2 = [rand(Uniform(0.1, 0.7)) for i in 1:50]
println(v2)
v3 = [rand(Uniform(0.5, 4.0)) for i in 1:50]
println(v3)

result1 = rand(Uniform(0.0, 1.0), 50)
result1 .= v1./v2 .+ 3.8 .*v3 .+ 5.0 .* log.((v1 .^2) ./v3)
println(result1)

result2 = rand(Uniform(0.0, 1.0), 50)
result2 .= 0.8 .* v1./v2 .+ 3.5 .*v3 .+ log.(5.0 .* (v1 .^3) ./v3)
println(result2)

df1 = DataFrame(v1 = v1, v2 = v2, v3 = v3, y = result1)
df2 = DataFrame(v1 = v1, v2 = v2, v3 = v3, y = result2)
CSV.write("./data/synthetic/synthetic_constructDimless_wo_noise.csv", df1)
CSV.write("./data/synthetic/synthetic_freeConst_wo_noise.csv", df2)

[8.639100743085532, 8.435773319398226, 7.387577239368664, 7.828858141115962, 7.100571656644625, 7.566810406168052, 7.656609035548126, 8.491539962314048, 5.994987560033631, 7.897075969933256, 7.3469249838980115, 5.455899645673138, 5.93477448543663, 6.449918203956688, 5.021352702157164, 9.827155255389421, 8.295750548552828, 5.482726233673562, 6.60605207862843, 7.417140072002406, 6.71308707641556, 6.946315848753331, 9.268353631865184, 8.62383953943768, 7.0275393032166855, 5.1289320022521725, 9.903909504031809, 7.907318372017803, 9.122549715645501, 5.114747346590121, 9.401582128153942, 5.833715618300463, 7.8342719084156585, 8.641561098026251, 9.401432783663976, 8.507111560671571, 7.960355378781964, 5.226129176826111, 7.630580449442256, 6.631064996025454, 7.707059415927103, 7.974753171314172, 8.089284376035124, 8.89761560207787, 9.996948691375101, 5.984784760550996, 5.232083964911804, 6.407905114094209, 5.1295027613499595, 5.168976780391028]
[0.5679095062860513, 0.6912906058116438, 0.136717

[42.83945875149891, 37.766663977069584, 77.95256870666596, 89.68925198778459, 35.04996709223559, 37.534046330079335, 51.455647772214284, 45.966673822249206, 44.974460864648165, 37.40352419313041, 47.18612451703867, 29.047315422640807, 42.069812864828606, 47.29182383477866, 36.752906367875646, 56.54422074762429, 39.55426860269567, 39.1111716431156, 36.84194074771363, 40.60068035744369, 42.29090483107191, 34.66975793929434, 46.90563863490209, 43.40112002272403, 41.10985147900459, 39.21966442990718, 45.7568623106045, 48.14061192762043, 76.79027483101773, 36.96840691039222, 40.43268703595754, 33.40536782343995, 48.48261751046793, 46.528282031867356, 103.51240358854284, 37.71533036794482, 86.56226047944892, 35.94971731413395, 38.49894611954305, 57.799085435301286, 43.094023764190226, 50.442562301961445, 37.96208637467929, 43.404700341800755, 46.752892683429685, 35.06239828383823, 30.182661709025563, 45.89475599038577, 32.339968344709774, 58.1409445601387]


[29.8812120589667, 20.720636190147907, 56.601140715838966, 66.15344131707741, 18.884785171081685, 24.919439497782673, 34.195335448589894, 32.600121458638, 31.184889183322184, 24.84706199574328, 34.787058444468656, 19.287908931126996, 31.889729713455438, 34.12421558967427, 24.648945290835222, 39.04734098624683, 26.477057701407656, 30.132289083422954, 26.534517844956383, 29.489615842670325, 31.073981916306238, 18.032935161822667, 32.90189343482128, 24.68599817509056, 25.23031992719135, 27.79181935870347, 32.43548757223551, 33.45896546847175, 57.31701836474291, 28.635111726307553, 24.79397567049592, 23.186862938552835, 29.4007336351745, 34.0426933363112, 77.22802615395952, 22.304482757820526, 64.53604669034873, 24.543774683794354, 24.344459173990305, 42.44659038422836, 29.327735058542633, 34.688343727774004, 25.231707992323635, 29.39279553087711, 31.89905329677596, 22.647796212927883, 18.946766040022723, 34.625386403060276, 18.767889954111112, 39.020044064347765]


"./data/synthetic/synthetic_freeConst_wo_noise.csv"

In [24]:

df1 = DataFrame(v1 = v1, v2 = v2, v3 = v3, y = result1)
df2 = DataFrame(v1 = v1, v2 = v2, v3 = v3, y = result2)
CSV.write("./data/synthetic/synthetic_constructDimless_generated_wo_noise.csv", df1)
CSV.write("./data/synthetic/synthetic_freeConst_generated_wo_noise.csv", df2)

"./data/synthetic/synthetic_freeConst_generated_wo_noise.csv"

In [25]:
scale = 0.01
result1_noise = result1 .+ scale * randn(length(result1)) .* result1
result2_noise = result2 .+ scale * randn(length(result2)) .* result2

df1 = DataFrame(v1 = v1, v2 = v2, v3 = v3, y = result1_noise)
df2 = DataFrame(v1 = v1, v2 = v2, v3 = v3, y = result2_noise)
CSV.write("./data/synthetic/synthetic_constructDimless_generated_001noise.csv", df1)
CSV.write("./data/synthetic/synthetic_freeConst_generated_001noise.csv", df2)

"./data/synthetic/synthetic_freeConst_generated_001noise.csv"

In [29]:
using DataFrames    
using CSV
df = CSV.read("./data/singleParticle/train_instance1.csv", DataFrame)
#println(df)
CSV.write("./data/singleParticle/train_instance1_generated_wo_noise.csv", df)

result1 = df[:, end]
println(result1)
scale = 0.02
result1_noise = result1 .+ scale * randn(length(result1)) .* result1
println(result1_noise)
df[:, end] = result1_noise
CSV.write("./data/singleParticle/train_instance1_generated_002noise.csv", df)


[0.6187493659814317, 0.6126805006221971, 0.6076243536078588, 0.603599328462258, 0.6005326939153286, 0.5982699927959596, 0.5966152781296071, 0.5953934779895733, 0.5945061653679011, 0.5939449790636564,

 0.59375, 0.5939449790636564, 0.594506165367901, 0.5953934779895733, 0.5966152781296071, 0.5982699927959594, 0.6005326939153286, 0.6035993284622581, 0.6076243536078587, 0.612680500622197, 0.6187493659814319, 0.5879668764401439, 0.579192962449677, 0.5714406178630712, 0.5647599626915963, 0.5590814144382346, 0.5542314345783336, 0.5500105186160446, 0.5463214212263843, 0.5432801633676876, 0.5412090753194853, 0.5404663923182442, 0.5412090753194851, 0.5432801633676876, 0.5463214212263845, 0.5500105186160443, 0.5542314345783336, 0.5590814144382346, 0.5647599626915962, 0.5714406178630711, 0.579192962449677, 0.5879668764401436, 0.554342775915129, 0.5420340355824195, 0.5306363723098183, 0.5202175653444961, 0.5106674553953875, 0.5017155448006387, 0.4930847855989481, 0.48478774569904226, 0.47741946683437647, 0.472150168551357, 0.47021484375, 0.4721501685513571, 0.4774194668343764, 0.48478774569904215, 0.49308478559894786, 0.5017155448006387, 0.5106674553953875, 0.5202175653444961, 0.530636372309818

, 0.4509748712981043, 0.4721279893523801, 0.48404318594339824, 0.49468370377584614, 0.5120584142331076, 0.46667864615652205, 0.44423130101464864, 0.43297393569236453, 0.4138077502366303, 0.3857875490488579, 0.3582154057801275, 0.3414432023570368, 0.29778569790583015, 0.2711230132846515, 0.24723074015374003, 0.23437878583771857, 0.2392042264593782, 0.27420918552954127, 0.3076873405215047, 0.3418676114425421, 0.3612321965939359, 0.3937193074642221, 0.41112966606457585, 0.4519870900264896, 0.4662301073722759, 0.47622380655361324, 0.43581992357972854, 0.4250504647264728, 0.39041644805413156, 0.3539008096644368, 0.3260916274904214, 0.2764768349975572, 0.24803920742455682, 0.17165320238223927, 0.09281164166433718, 0.02859963379843972, 3.412207227748653e-16, 0.027176591347382203, 0.09685359208395382, 0.17287349216677753, 0.2329165921316248, 0.27923519960884685, 0.31936687396935026, 0.3440421890202803, 0.38892366608320766, 0.4061167119842678, 0.441908854558429, 0.40778907508691326, 0.366311361

"./data/singleParticle/train_instance1_generated_002noise.csv"